# Test MTCS

This notebook contains some examples on how to use and test the `MTCS` class for high level control.

One benefit of using the `MTCS` class instead of instantiating the remotes individually is that you get all the remotes for the components in addition to the high level tasks.

For now the MTCS only have limited functionality implemented and they are not well tested and sanctioned. Some generic behavior (like `enable`) also have some limitations as some components are not adhering to the architecture. For instance, M1M3 is not sending the `settingVersions` event with information about configurations but it requires a configuration to be specified (e.g. `Default`). This is worked around by providing the configuration explicitly when calling the `enable` method, as one can see below. 

Contact Tiago Ribeiro for help: `tribeiro` on slack. 

In [ ]:
import sys
import asyncio
import logging
from lsst.ts import salobj

from lsst.ts.observatory.control.maintel import MTCS

In [ ]:
stream_handler = logging.StreamHandler(sys.stdout)

logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [ ]:
mtcs = MTCS()

In [ ]:
await mtcs.start_task

In [ ]:
mtcs.components

In [ ]:
mtcs.check.dome = False
mtcs.check.mtdometrajectory = False

In [ ]:
await mtcs.rem.hexapod_1.evt_heartbeat.next(flush=True, timeout=5)

In [ ]:
await mtcs.rem.hexapod_2.evt_heartbeat.next(flush=False, timeout=5)

In [ ]:
await mtcs.rem.newmtmount.evt_heartbeat.next(flush=True, timeout=10)

In [ ]:
await mtcs.rem.newmtmount.evt_heartbeat.next(flush=True)

In [ ]:
await asyncio.sleep(5.)
for comp in mtcs.components:
    if not getattr(mtcs.check, comp):
        continue
    
    try:
        state = await mtcs.get_state(comp)
        print(f"{comp}: {state!r}")
    except asyncio.TimeoutError:
        print(f"Failed to get status for {comp}.")
        pass

Some CSCs like the Rotator and Hexapod, when transition to `FAULT` state will only transition back to `STANDBY` if a `clearError` command is sent. 

The cell bellow is commented so it will not run in normal circumstances, but it you note that the Rotator (or any of the hexapods) are in `FAULT` state you can uncomment/edit it to fix the problem.

In [ ]:
# await mtcs.rem.rotator.cmd_clearError.start()

In [ ]:
await mtcs.enable({'newmtmount': '',
 'mtmount': '',
 'mtptg': '',
 'mtaos': 'default',
 'mtm1m3': 'Default',
 'mtm2': '',
 'hexapod_1': '',
 'hexapod_2': '',
 'rotator': '',
 'dome': '',
 'mtdometrajectory': ''})

In [ ]:
await mtcs.rem.newmtmount.cmd_enableCameraCableWrapTracking.start()

## Slew to a circumpolar target

This next cell will slew to `HD 195875` which is a circumpolar target, hence, always visible at the observatory. One detail to keep in mind is that the rotator position might have to be tweaked. If the command fails with an exception like "rotator position out of range" try changing to another value.

In [ ]:
await mtcs.slew_object("HD 185975", rot_sky=45.)

The next target is also "HD 185975", but slewing to the coordinates.

In [ ]:
try:
    await mtcs.slew_icrs(ra="20 28 18.7402", dec="-87 28 19.938", rot_sky=120., stop_before_slew=False)
except RuntimeError:
    print("Error. wainting 30s")
    await asyncio.sleep(30)

## Slew to a position in the sky at declination zero (north of the observatory).

The next cell will slew to a field at declination zero. It makes sure it can always slew by getting the current Local Sidereal Time from the pointing.


In [ ]:
time_and_date = await mtcs.rem.mtptg.tel_timeAndDate.aget()
ra = Angle(time_and_date.lst, unit=u.hourangle)

try:
    await mtcs.slew_icrs(ra=ra, dec="00:00:00", rot_sky=-135., stop_before_slew=False)
except RuntimeError:
    print("Error. wainting 30s")
    await asyncio.sleep(30)